# Project Description
What goes here: updated project schedule (any changes in plans?). What will go into the final analysis? Any deviations from the initial plan.

# Data Description
What goes here: elaborated data description, acquisition, cleaning. Any deviations from the initial project plan?

# Ethical Data Concerns


# Methods


# Preliminary Results

In [ ]:
# Import useful modules
import pyodbc
import pandas as pd
import os
import warnings

# NOTE: dates are formatted month/day/year
#       time is in UTC zulu format
#       wind velocity and gust speed are in knots
#       it doesn't look like wx_int_precip is used
#       the amount of fuel on board is in gallons

def Get_Aviation_Data(db_name):
    """Retrieves desired data from the Microsoft Access Databases 'avall' and 'Pre2008' and returns it in a pandas dataframe."""

    # Determine the current directory
    current_dir = os.getcwd()

    # Connect to the database
    conn_str = (r'DRIVER={Microsoft Access Driver (*.mdb, *.accdb)};'
                fr'DBQ={current_dir}\{db_name};')
    conn = pyodbc.connect(conn_str)

    if db_name == 'avall.mdb' or db_name == 'Pre2008.mdb':

        # Read sql from the database that uses JOIN to get the columns we want
        #main_df = pd.read_sql("""
        #                        SELECT e.ev_date, e.ev_time, e.latitude, e.longitude, e.mid_air, 
        #                            e.on_ground_collision, e.light_cond, e.sky_ceil_ht, 
        #                            e.wind_vel_kts, e.gust_kts, e.altimeter, e.wx_int_precip, 
        #                            e.ev_highest_injury, a.fuel_on_board, a.acft_model, 
        #                            a.acft_make, a.damage, i.inj_person_count, o.Altitude,
        #                            f.finding_description
        #                        FROM events e
        #                        LEFT JOIN aircraft a ON e.ev_id = a.ev_id
        #                        LEFT JOIN injury i ON a.ev_id = i.ev_id
        #                        LEFT JOIN Occurrences o ON a.ev_id = o.ev_id
        #                        LEFT JOIN Findings f ON a.ev_id = f.ev_id
        #                        """, conn)

        # Make arrays of the columns we want from the "events" and "aircraft" tables
        events_columns = ['ev_date', 'ev_time', 'latitude', 'longitude', 'mid_air', 'on_ground_collision', 'light_cond', 'sky_ceil_ht', 'wind_vel_kts', 'gust_kts', 'altimeter', 'wx_int_precip', 'ev_highest_injury']

        if db_name == 'avall.mdb':
            aircraft_columns = ['fuel_on_board', 'acft_model', 'acft_make', 'damage']
        else:
            aircraft_columns = ['acft_model', 'acft_make', 'damage'] 
            # NOTE: Pre2008 doesn't have 'fuel_on_board'

        #  Read in the entire tables for tables that have more than one column (reading in only the columns needed didn't work)
        events_df = pd.read_sql("SELECT * FROM events", conn)
        aircraft_df = pd.read_sql("SELECT * FROM aircraft", conn)

        # Read in the columns from the tables that only have one or two columns
        inj_person_count = pd.read_sql("SELECT inj_person_count FROM injury", conn)
        altitude = pd.read_sql("SELECT Altitude FROM Occurrences", conn)
        finding_description = pd.read_sql("SELECT finding_description FROM Findings", conn)

        # Keep only the desired columns
        events_df = events_df[events_columns]
        aircraft_df = aircraft_df[aircraft_columns]

        # Join the dataframes
        dataframes = events_df.join(aircraft_df)

        # Add the columns that were read in on their own
        dataframes['inj_person_count'] = inj_person_count
        dataframes['altitude'] = altitude
        dataframes['finding_description'] = finding_description

    else:
        # Make arrays of the columns we want from the 'tblFirstHalf', 'tblSecondHalf', and 'ct_Pre1982' tables
        tblFirstHalf_columns = ['DATE_OCCURRENCE', 'TIME_OCCUR', 'LOCATION','LIGHT_COND', 'ALT_OF_OCC', 'ACFT_MODEL', 'ACFT_MAKE', 'ACFT_ADAMG', 'ACCIDENT_TYPE1', 'ACCIDENT_TYPE2',
                                'GRAND_TOTAL_FATAL', 'GRAND_TOTAL_SERIOUS', 'GRAND_TOTAL_MINOR', 'GRAND_TOTAL_NONE']
        tblSecondHalf_columns = ['CEILING', 'WIND_VELOCITY', 'PRECIPITATION', 'CAUSE'] 
        ct_Pre1982_names = ['ACCIDENT_TYPE', 'CAUSE_FACTOR', 'OPERATION_PHASE', 'PRECIPITATION', 'ACFT_ADAMG', 'LIGHT_COND', 'LOCAT_STATE_TERR']
        tblOcurrances_columns = ['Altitude', 'PhaseOfFlight']
        # NOTE: the closest I found to 'wx_int_precip' was 'Precipitation' (Precipitation from the ct_Pre1982 table has the codes),
        #       the closest I found to 'latitude' and 'longitude' is 'location' (locat_state_terr has codes), 
        #       the closest I found to 'ev_highest_injury' and 'inj_person_count' were actually four different columns: 'grand_total_fatal', 'grand_total_serious', 'grand_total_minor', 'grand_total_none' 
        #       the closest I found to 'finding_description' was three different columns: 'Accident_Type1' & 'Accident_Type2' (Accident_Type has codes) and 'Cause' (cause_factor has codes)
        #       the closest I found to 'on_ground_collision' is 'PhaseOfFlight' (operation_phase has the codes)
        #       didn't find anything that matched 'fuel_on_board', 'altimeter', or 'gust_kts'

        # Read in each table
        tblFirstHalf = pd.read_sql("SELECT * FROM tblFirstHalf", conn)
        tblSeconfHalf = pd.read_sql("SELECT * FROM tblSecondHalf", conn)
        ct_Pre1982 = pd.read_sql("SELECT * FROM ct_Pre1982", conn)
        tblOcurrances = pd.read_sql("SELECT * FROM tblOccurrences", conn)

        # set the index to be the 'Name' column, that way my code works
        ct_Pre1982.set_index('Name', inplace = True)

        # Keep only the desired columns
        tblFirstHalf = tblFirstHalf[tblFirstHalf_columns]
        tblSeconfHalf = tblSeconfHalf[tblSecondHalf_columns]
        ct_Pre1982 = ct_Pre1982.loc[ct_Pre1982_names]
        tblOcurrances = tblOcurrances[tblOcurrances_columns]

        # Join the dataframes
        main_df1 = tblFirstHalf.join(tblSeconfHalf)
        main_df = main_df1.join(tblOcurrances)

        # call the main dataframe and the codes dataframe 'dataframes'
        dataframes = [main_df, ct_Pre1982]

    # Return the dataframe
    return dataframes

# Read in the data and ignore the warnings (they're annoying)
with warnings.catch_warnings(action="ignore"):
    df_pre_1982, df_codes_pre_1982 = Get_Aviation_Data("PRE1982.mdb")
    df_1982_to_2008 = Get_Aviation_Data("Pre2008.mdb")
    df_2008_to_2025 = Get_Aviation_Data("avall.mdb")

# Print some of each dataframe with plenty of room between the prints so I can tell them apart
print(df_codes_pre_1982)
print('\n\n\n\n')
print(df_pre_1982.head())
print('\n\n\n\n')
print(df_1982_to_2008.head())
print('\n\n\n\n')
print(df_2008_to_2025.head())

                 Code                             Meang
Name                                                   
ACCIDENT_TYPE       0           HAIL DAMAGE TO AIRCRAFT
ACCIDENT_TYPE       1                  LIGHTNING STRIKE
ACCIDENT_TYPE       2                  EVASIVE MANEUVER
ACCIDENT_TYPE       3  UNCONTROLLED ALTITUDE DEVIATIONS
ACCIDENT_TYPE       4                          DITCHING
...               ...                               ...
LOCAT_STATE_TERR   95        OTHER-INTERNATIONAL WATERS
LOCAT_STATE_TERR    F                 FOREIGN COUNTRIES
LOCAT_STATE_TERR    S                     UNITED STATES
LOCAT_STATE_TERR    T     U.S.TERRITORIES & POSSESSIONS
LOCAT_STATE_TERR    W              INTERNATIONAL WATERS

[1184 rows x 2 columns]





  DATE_OCCURRENCE TIME_OCCUR         LOCATION LIGHT_COND ALT_OF_OCC  \
0      1962-07-23       1530          ROME,NY          B       None   
1      1962-09-11       1544   CLEVELAND,OHIO          B       None   
2      1962-08-30       0927 

#### claude ai conversations

**Question:** <br>I run comment#Import useful modules
import pyodbc
import pandas as pd
import os
comment#Determine the paths to the files
current_dir = os.getcwd()
path_older_data = fr"{current_dir}\Pre2008.mdb;"
comment#Read in desired data from the file with data 2008 and more recent
comment#Connect to the database - NOTE: You must NOT have the database open
conn_str = (r'DRIVER={Microsoft Access Driver (*.mdb, *.accdb)};'
            fr'{current_dir}\avall.mdb;')
conn = pyodbc.connect(conn_str)
comment#Read in the entire tables for tables that have more than one column (reading in only the columns needed didn't work)
events_df = pd.read_sql("SELECT * FROM events", conn)
aircraft_df = pd.read_sql("SELECT * FROM aircraft", conn)
comment#Read in the columns from the tables that only have one or two columns
inj_person_count = pd.read_sql("SELECT inj_person_count FROM injury", conn)
altitude = pd.read_sql("SELECT Altitude FROM Occurrences", conn)
finding_description = pd.read_sql("SELECT finding_description FROM Findings", conn)
comment#Make arrays of the columns we want from the "events" and "aircraft" tables
events_columns = ['ev_date', 'ev_time', 'latitude', 'longitude', 'mid_air', 'on_ground_collision', 'light_cond', 'sky_ceil_ht', 'wind_vel_kts', 'gust_kts', 'altimeter', 'wx_int_precip', 'ev_highest_injury']
aircraft_columns = ['fuel_on_board', 'acft_model', 'acft_make', 'damage']
comment#Keep only the desired columns
events_df = events_df[events_columns]
aircraft_df = aircraft_df[aircraft_columns]
comment#Concatonate the dataframes
df_2008 = events_df.join(aircraft_df)
comment#add the columns that were read in on their own
df_2008['inj_person_count'] = inj_person_count
df_2008['altitude'] = altitude
df_2008['finding_description'] = finding_description
comment#Print the main dataframe
print(df_2008.head())
and get Error: ('01S00', '[01S00] [Microsoft][ODBC Driver Manager] Invalid connection string attribute (0) (SQLDriverConnect)')<br>
**Answer:** <br>You're missing the 'DBQ=' parameter before the file path. Try this: pythonCopyconn_str = (r'DRIVER={Microsoft Access Driver (*.mdb, *.accdb)};'fr'DBQ={current_dir}\avall.mdb;')<br>
**Action Taken:** <br>I replaced conn_str = (r'DRIVER={Microsoft Access Driver (*.mdb, *.accdb)};'fr'{current_dir}\avall.mdb;') 
with (r'DRIVER={Microsoft Access Driver (*.mdb, *.accdb)};'fr'DBQ={current_dir}\avall.mdb;')<br><br><br>

**Question:** <br>Why is this code not working?
comment#Import useful modules
import pyodbc
import pandas as pd
comment#Connect to the database - NOTE: You must NOT have the database open
conn_str = (r'DRIVER={Microsoft Access Driver (*.mdb, *.accdb)};'
            r'DBQ=C:\Users\erken\Desktop\School Stuff\Intro to Data Science\Intro_To_Data_Science\avall.mdb;')
conn = pyodbc.connect(conn_str)
comment#Read in desired data from the file with data 2008 and more recent
comment#Read in the entire tables for tables that have more than one column (reading in only the columns needed didn't work)
events_df = pd.read_sql("SELECT * FROM events", conn)
aircraft_df = pd.read_sql("SELECT * FROM aircraft", conn)
comment#Read in the columns from the tables that only have one or two columns
inj_person_count = pd.read_sql("SELECT inj_person_count FROM injury", conn)
altitude = pd.read_sql("SELECT Altitude FROM Occurrences", conn)
finding_description = pd.read_sql("SELECT finding_description FROM Findings", conn)
comment#Make arrays of the columns we want from the "events" and "aircraft" tables
events_columns = ['ev_date', 'ev_time', 'latitude', 'longitude', 'mid_air', 'on_ground_collision', 'light_cond', 'sky_ceil_ht', 'wind_vel_kts', 'gust_kts', 'altimeter', 'wx_int_precip', 'ev_highest_injury']
aircraft_columns = ['fuel_on_board', 'acft_model', 'acft_make', 'damage']
print(events_df.columns)
comment#If a column of events_df is not in events_columns, then drop the column from events_df
for column in events_df.columns:
    if not column in events_columns:
        events_df.drop(column)<br>
**Answer:** <br>The main issue is that your drop columns code isn't working because .drop() doesn't modify the DataFrame in place by default. You need to either:
Use inplace=True:
pythonCopyfor column in events_df.columns:
    if column not in events_columns:
        events_df.drop(column, inplace=True)
Or assign the result back:
pythonCopyfor column in events_df.columns:
    if column not in events_columns:
        events_df = events_df.drop(column, axis=1)
A better approach would be:
pythonCopyevents_df = events_df[events_columns]
aircraft_df = aircraft_df[aircraft_columns]<br>
**Action Taken:** <br>replaced 
comment#If a column of events_df is not in events_columns, then drop the column from events_df
for column in events_df.columns:
    if not column in events_columns:
        events_df.drop(column)
with
comment#Keep only the desired columns
    events_df = events_df[events_columns]
    aircraft_df = aircraft_df[aircraft_columns]<br><br><br>

**Question:** <br>df = pd.read_sql('select column1, column2  from table_name', conn) 
is that the right syntax for selecting different entries from a table?<br>
**Answer:** <br>Yes, your syntax for reading from a SQL table using pandas is correct:
pythonCopydf = pd.read_sql('select column1, column2 from table_name', conn)
This code will execute the SQL query and store the results in a pandas DataFrame. The conn object should be your database connection.<br>
**Action Taken** <br>ask the following question: <br><br><br>

**Question:** <br>I run # Import useful modules
import pyodbc
import pandas as pd
comment#Connect to the database - NOTE: You must NOT have the database open
conn_str = (r'DRIVER={Microsoft Access Driver (*.mdb, *.accdb)};'
            r'DBQ=C:\Users\erken\Desktop\School Stuff\Intro to Data Science\Intro_To_Data_Science\avall.mdb;')
conn = pyodbc.connect(conn_str)
comment#Read in desired data from the file with data 2008 and more recent
comment#Desired entries are ev_date, ev_time, latitude, longitude, mid_air, on_ground_collision, light_cond, sky_ceil_ht, wind_vel_kts, gust_kt, 
comment#altimeter, wx_int_precip, fuel_on_board, acft_model, acft_make, damage, Altitude, inj_person_count, and finding_description
df_2008 = pd.read_sql('SELECT ev_date, ev_time, latitude, longitude, mid_air, on_ground_collision, light_cond, sky_ceil_ht, '
                      'wind_vel_kts, gust_kt, altimeter, wx_int_precip, ev_highest_injury FROM events', conn)
I get the error Error: ('07002', '[07002] [Microsoft][ODBC Microsoft Access Driver] Too few parameters. Expected 1. (-3010) (SQLExecDirectW)') <br>
**Answer:** <br>The error message indicates you're missing a parameter that the query expects. This typically happens when your SQL contains a parameter placeholder (often marked with ? or @parameter) but you haven't provided the corresponding value.
Looking at your query, I don't see any explicit parameter placeholders, but Access might be interpreting something in your query as a parameter. Some possible fixes:
Check for special characters in column names that might need to be escaped (try enclosing column names in square brackets):
pythonCopydf_2008 = pd.read_sql('SELECT ev_date, ev_time, latitude, longitude, mid_air, on_ground_collision, [light_cond], [sky_ceil_ht], '
                     '[wind_vel_kts], [gust_kt], altimeter, wx_int_precip, ev_highest_injury FROM events', conn)
Try using a parameterized query explicitly if you need one:
pythonCopydf_2008 = pd.read_sql('SELECT * FROM events WHERE some_column = ?', conn, params=('parameter_value',))
Check if any column names contain reserved words that need bracketing.<br>
**Action Taken:** <br>ask the following question: <br><br><br>

**Question:** <br>Here are the SQL tables 
and here is what the file shows (I inserted images of the database schema and the actual program) <br>
**Answer:** <br>Based on the images, I can see that you need to use the dt_events table since it's selected in image 2, and I can see all the available fields in image 1.
Try this corrected query with square brackets around column names:
pythonCopydf_2008 = pd.read_sql('SELECT [ev_date], [ev_time], [latitude], [longitude], [mid_air], [on_ground_collision], [light_cond], [sky_ceil_ht], [wind_vel_kts], [gust_kts], [altimeter], [wx_int_precip], [ev_highest_injury] FROM [dt_events]', conn)
Key corrections:
Added square brackets around all column names and table name
Notice I changed gust_kt to gust_kts based on image 1
If issues persist, try using a cursor approach instead:
pythonCopycursor = conn.cursor()
cursor.execute('SELECT [ev_date], [ev_time], [latitude], [longitude], [mid_air], [on_ground_collision], [light_cond], [sky_ceil_ht], [wind_vel_kts], [gust_kts], [altimeter], [wx_int_precip], [ev_highest_injury] FROM [dt_events]')
df_2008 = pd.DataFrame.from_records(cursor.fetchall(), columns=[col[0] for col in cursor.description]) <br>
**Action Taken:** <br>ask the following question: <br><br><br>

**Question:** <br>I tried both and still get Error: ('07002', '[07002] [Microsoft][ODBC Microsoft Access Driver] Too few parameters. Expected 13. (-3010) (SQLExecDirectW)') <br>
**Answer:** <br>The "Too few parameters. Expected 13" error suggests MS Access is still interpreting column names as parameters. Let's try more aggressive approaches:
Try with double brackets:
pythonCopydf_2008 = pd.read_sql("SELECT [[ev_date]], [[ev_time]], [[latitude]], [[longitude]], [[mid_air]], [[on_ground_collision]], [[light_cond]], [[sky_ceil_ht]], [[wind_vel_kts]], [[gust_kts]], [[altimeter]], [[wx_int_precip]], [[ev_highest_injury]] FROM [[dt_events]]", conn)
Try a simpler query first to verify basic connectivity:
pythonCopytest_df = pd.read_sql("SELECT [ev_id] FROM [dt_events]", conn)
Or try retrieving all columns, which sometimes avoids parameter issues:
pythonCopydf_2008 = pd.read_sql("SELECT * FROM [dt_events]", conn)<br>
**Action Taken:**<br> Try those too, settle on getting all the columns and then filtering afterwards<br><br><br>

# Completed Milestones


# Methods Milestones


# Summary